# Logging
This tutorial demonstrates how to use the `DataLogger` from `src/gamecore/utils` to systematically log and load objects and simulation data, which is essential for reproducible experiment.

### 1. Imports

To access the files in `src/` from within this notebook, we need to add the root directory to the Python path.

In [ ]:
import sys
import os
# Get the absolute root path of this repository
root_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
# Add it to the Python path
if root_path not in sys.path:
    sys.path.insert(0, root_path)
    
from src.gamecore import (
    DataLogger,
    make_random_lq_game
)
%matplotlib inline

SEED = 999

## 2. Creating a DataLogger

An `DataLogger` manages file-based logging of objects and metadata.  
It requires:

- `base_dir`: The folder in which the experiment will be stored (absolute or relative)
- `experiment_name`: (Optional) A custom name for the experiment folder  
  → If not provided, a timestamp-based name is generated automatically.

Since we're in a Jupyter notebook, it's best to use an **absolute path** for `base_dir`, e.g., pointing to `data/`.


In [ ]:
logger = DataLogger(
    base_dir=os.path.join(root_path, "data"),
    folder_name="tutorial_logging"
)
logger.summarize()

## 3. Example: Logging a GradientPlay Simulation

### a) Creating a simulated LQGame

We use a utility function from `src/utils` to generate a random linear-quadratic game.  
This function supports various customization options, see [src/utils/factories/game_factory.py](../../src/utils/factories/game_factory.py) for all options.

In [ ]:
r_jj = "free"
r_jk = "zero"
game = make_random_lq_game(seed=SEED, cost_r_jj=r_jj, cost_r_jk=r_jk, learning_rate=0.01)

### b) Logging the Simulation

Nearly all objects support `log()` and `load()` methods for structured saving.  
In particular:

- **High-level objects recursively log their subcomponents**
- The `prefix` argument enables hierarchical file names  
  (e.g., the game logs the system as `"system_*"`)

The following cell logs the complete `LQGame` object.


In [ ]:
game.log(logger, prefix="game_")
print(f"Experiment logged to: {logger.dir}")

You can attach custom metadata to the experiment by passing a dictionary to `logger.log_metadata()`.

This is especially useful for saving parameters for factory methods like seed, cost-specifications, etc.


In [ ]:
logger.log_metadata({
    "seed": SEED,
    "r_jj": r_jj,
    "r_jk": r_jk,
})

Using the `load` method, we can retrieve the information:

In [ ]:
loaded_game = game.load(logger, prefix="game_") 
loaded_seed = logger.load_metadata_entry("seed")
loaded_r_jj = logger.load_metadata_entry("r_jj")
loaded_r_jk = logger.load_metadata_entry("r_jk")

print(type(loaded_game) == type(game))
print(loaded_seed == SEED)
print(loaded_r_jj == r_jj)
print(loaded_r_jk == r_jk)
print(loaded_game.system.A == game.system.A)
print(loaded_game.system.Bs[0] == game.system.Bs[0])
print(loaded_game.players[0].strategy.K == game.players[0].strategy.K)
# ... and so on